In [ ]:
#如何生成一个简单爬虫
#import requests
#import csv
#import re
#from pyquery import PyQuery
#上述四个为常见的工具，建议无脑贴

In [1]:
#1.寻找网址规律，简单的可以直接看，稍微复杂的用F12开发者工具
#简单网址
def generate_urls(limit=10):
    """
    生成网站的url列表
    limit: 最大可采集页数，默认是50。豆瓣小说允许的最大采集不得超过50页。
    """
    urls = []
    template = 'https://book.douban.com/tag/小说?start={param}&type=T'

    for p in range(1, limit+1):
        url = template.format(param=(p-1)*20)
        urls.append(url)
    return urls

generate_urls()

['https://book.douban.com/tag/小说?start=0&type=T',
 'https://book.douban.com/tag/小说?start=20&type=T',
 'https://book.douban.com/tag/小说?start=40&type=T',
 'https://book.douban.com/tag/小说?start=60&type=T',
 'https://book.douban.com/tag/小说?start=80&type=T',
 'https://book.douban.com/tag/小说?start=100&type=T',
 'https://book.douban.com/tag/小说?start=120&type=T',
 'https://book.douban.com/tag/小说?start=140&type=T',
 'https://book.douban.com/tag/小说?start=160&type=T',
 'https://book.douban.com/tag/小说?start=180&type=T']

In [14]:
#复杂网址
def generate_url(stockcode, beginDate, endDate):
    """
    生成年报搜索页的url
    注意，开始日期与结束日期之间要小于三年
    stockcode： 股票代码字符串 比如'600000'
    beginDate： 开始日期字符串，如'2017-10-05'
    endDate： 结束日期字符串，如'2020-04-05'
    """
    template = 'http://query.sse.com.cn/security/stock/queryCompanyBulletin.do?jsonCallBack=jsonpCallback10470&isPagination=true&productId={stockcode}&securityType=0101%2C120100%2C020100%2C020200%2C120200&reportType2=DQBG&reportType=&beginDate={beginDate}&endDate={endDate}&pageHelp.pageSize=25&pageHelp.pageCount=50&pageHelp.pageNo=1&pageHelp.beginPage=1&pageHelp.cacheSize=1&pageHelp.endPage=5&_=1588153366632'
    url = template.format(stockcode=stockcode, beginDate=beginDate, endDate=endDate)
    return url

stockcode = '600000'
beginDate = '2017-10-05'
endDate = '2020-04-20'

url = generate_url(stockcode, beginDate, endDate)
url


'http://query.sse.com.cn/security/stock/queryCompanyBulletin.do?jsonCallBack=jsonpCallback10470&isPagination=true&productId=600000&securityType=0101%2C120100%2C020100%2C020200%2C120200&reportType2=DQBG&reportType=&beginDate=2017-10-05&endDate=2020-04-20&pageHelp.pageSize=25&pageHelp.pageCount=50&pageHelp.pageNo=1&pageHelp.beginPage=1&pageHelp.cacheSize=1&pageHelp.endPage=5&_=1588153366632'

In [ ]:
#按找以下顺序排除问题 headers、cookies、referer

In [11]:
#2.获取源代码，简单网址
import requests 

def get_html(url):
    """
    获取url对应的html数据(网页源代码字符串数据)，返回html数据
    url: 网页的url
    """
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.122 Safari/537.36'}
    #headers用于反爬
    resp = requests.get(url, headers=headers)
    html = resp.text
    return html

url = 'https://book.douban.com/tag/小说?start=0&type=T'
html = get_html(url)
html

'\n\n<!DOCTYPE html>\n<html lang="zh-cmn-Hans" class="ua-windows ua-webkit book-new-nav">\n<head>\n  <meta http-equiv="Content-Type" content="text/html; charset=utf-8">\n  <title>\n  豆瓣图书标签: 小说\n</title>\n  \n<script>!function(e){var o=function(o,n,t){var c,i,r=new Date;n=n||30,t=t||"/",r.setTime(r.getTime()+24*n*60*60*1e3),c="; expires="+r.toGMTString();for(i in o)e.cookie=i+"="+o[i]+c+"; path="+t},n=function(o){var n,t,c,i=o+"=",r=e.cookie.split(";");for(t=0,c=r.length;t<c;t++)if(n=r[t].replace(/^\\s+|\\s+$/g,""),0==n.indexOf(i))return n.substring(i.length,n.length).replace(/\\"/g,"");return null},t=e.write,c={"douban.com":1,"douban.fm":1,"google.com":1,"google.cn":1,"googleapis.com":1,"gmaptiles.co.kr":1,"gstatic.com":1,"gstatic.cn":1,"google-analytics.com":1,"googleadservices.com":1},i=function(e,o){var n=new Image;n.onload=function(){},n.src="https://www.douban.com/j/except_report?kind=ra022&reason="+encodeURIComponent(e)+"&environment="+encodeURIComponent(o)},r=function(o){try{t.

In [15]:
#复杂网址
import requests

def get_json(url):
    """
    获取url（报告列表页的url）对应的json数据
    返回raw_json
    """
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.122 Safari/537.36',
              'Referer': 'http://www.sse.com.cn/disclosure/listedinfo/regular/'}
    resp = requests.get(url, headers=headers)#, cookies=cookies)
    raw_json = resp.text
    return raw_json

raw_json = get_json(url)
raw_json

'jsonpCallback10470({"BULLETIN_TYPE":null,"END_DATE":null,"SECURITY_CODE":null,"START_DATE":null,"TITLE":null,"beginDate":"2017-10-05","endDate":"2020-04-20","isNew":null,"isPagination":"true","jsonCallBack":"jsonpCallback10470","keyWord":null,"pageHelp":{"beginPage":1,"cacheSize":1,"data":[{"ADDDATE":"2019-10-29 18:41:25","BULLETIN_HEADING":"定期报告","BULLETIN_TYPE":"第三季度季报","BULLETIN_YEAR":"2019","INDEXCLASS":null,"OPERATION_SEQ":null,"PLAN_Date":null,"PLAN_Year":null,"ROWNUM":null,"ROWNUM_":null,"SECURITY_CODE":"600000","SECURITY_NAME":null,"SSEDATE":"2019-10-30","SSEDate":null,"SSETime":null,"SSETimeStr":null,"TITLE":"浦发银行2019年第三季度报告","URL":"\\/disclosure\\/listedinfo\\/announcement\\/c\\/2019-10-30\\/600000_2019_3.pdf","author":null,"book_Name":null,"bulletinHeading":null,"bulletinType":null,"bulletin_No":null,"bulletin_Type":null,"bulletin_Year":null,"category_A":null,"category_B":null,"category_C":null,"category_D":null,"chapter_No":null,"companyAbbr":null,"dispatch_Organ":null,"fi

In [12]:
#3.解析数据。简单网址
from pyquery import PyQuery
import re

def extract_bookinfos(html):
    """
    从网页中抽取出书的相关信息，以字典列表返回
    html: 网页字符串数据
    """
    bookinfos = []
    doc = PyQuery(html)
    for book in doc.items('.subject-item'):
        img = book('.pic a img').attr('src')
        book_name = book('.info h2 a').text()
        info = book('.info .pub').text().split('/')
        price = info[-1]
        year = info[-2]
        pub = re.sub('\s', '', info[-3])
        authors = ''.join(info[:-3])
        rating = book('.rating_nums').text()
        #rate_people = book('span[class=pl]')

        raw_people = book('.pl').text()
        rate_people = re.findall('\d+', raw_people)[0]

        desc = book('.info p').text()                      

        book_info = {'img':img,
                     'book_name':book_name,
                     'price':price,
                     'year':year,
                     'pub':pub,
                     'authors':authors,
                     'rating': rating,
                     'rate_people':rate_people,
                     'desc':desc
                    }
        
        bookinfos.append(book_info)
    return bookinfos

extract_bookinfos(html)

[{'img': 'https://img9.doubanio.com/view/subject/s/public/s34544956.jpg',
  'book_name': '太白金星有点烦',
  'price': ' 45.00',
  'year': ' 2023-6 ',
  'pub': '湖南文艺出版社',
  'authors': '马伯庸 ',
  'rating': '9.0',
  'rate_people': '80431',
  'desc': '太白金星李长庚最近有点烦。 天庭和西天联合推出了“西天取经”的重大项目，他受命策划九九八十一难，确保唐僧能安全走完流程，平稳取经成佛。老神仙本以为一切尽在掌...'},
 {'img': 'https://img3.doubanio.com/view/subject/s/public/s34327482.jpg',
  'book_name': '长安的荔枝',
  'price': ' 45.00元',
  'year': ' 2022-10 ',
  'pub': '湖南文艺出版社',
  'authors': '马伯庸 ',
  'rating': '8.5',
  'rate_people': '178147',
  'desc': '大唐天宝十四年，长安城的小吏李善德突然接到一个任务：要在贵妃诞日之前，从岭南运来新鲜荔枝。荔枝“一日色变，两日香变，三日味变”，而岭南距长安五千余里，山水迢...'},
 {'img': 'https://img1.doubanio.com/view/subject/s/public/s34734949.jpg',
  'book_name': '额尔古纳河右岸',
  'price': ' 32.00元',
  'year': ' 2019-6-1 ',
  'pub': '人民文学出版社',
  'authors': '迟子建 ',
  'rating': '9.1',
  'rate_people': '42931',
  'desc': '《额尔古纳河右岸》是一部描写鄂温克人生存现状及百年沧桑的长篇小说，展示了弱小民族在严酷的自然环境和现代文明的挤压下的顽强生命力和不屈不挠的民族精神，以及丰富...'},
 {'img': 'https://img

In [16]:
#复杂网址
import json

def extract_pdf_links(raw_json):
    """
    从raw_json中抽取出pdf报告的链接link，返回的是link列表
    """
    links = []
    jsonstring = raw_json.replace('jsonpCallback10470(', '')[:-1]
    raw_dict_data = json.loads(jsonstring)
    results = raw_dict_data['result']
    for result in results:
        link = 'http://www.sse.com.cn'+result['URL']
        links.append(link)
    return links
  
links = extract_pdf_links(raw_json)
links

['http://www.sse.com.cn/disclosure/listedinfo/announcement/c/2019-10-30/600000_2019_3.pdf',
 'http://www.sse.com.cn/disclosure/listedinfo/announcement/c/2019-08-24/600000_2019_z.pdf',
 'http://www.sse.com.cn/disclosure/listedinfo/announcement/c/2019-08-24/600000_2019_zzy.pdf',
 'http://www.sse.com.cn/disclosure/listedinfo/announcement/c/2019-04-30/600000_2019_1.pdf',
 'http://www.sse.com.cn/disclosure/listedinfo/announcement/c/2019-03-26/600000_2018_n.pdf',
 'http://www.sse.com.cn/disclosure/listedinfo/announcement/c/2019-03-26/600000_2018_nzy.pdf',
 'http://www.sse.com.cn/disclosure/listedinfo/announcement/c/2018-10-31/600000_2018_3.pdf',
 'http://www.sse.com.cn/disclosure/listedinfo/announcement/c/2018-08-30/600000_2018_z.pdf',
 'http://www.sse.com.cn/disclosure/listedinfo/announcement/c/2018-08-30/600000_2018_zzy.pdf',
 'http://www.sse.com.cn/disclosure/listedinfo/announcement/c/2018-04-28/600000_2017_n.pdf',
 'http://www.sse.com.cn/disclosure/listedinfo/announcement/c/2018-04-28/60

In [13]:
#4.存储数据。简单网址
import csv 

#新建csv
csvf = open('data/book_infos.csv', 'a+', encoding='utf-8', newline='')
fieldnames = ['book_name', 'price', 'year', 'pub', 'authors', 'rating', 'rate_people', 'desc', 'img']
writer = csv.DictWriter(csvf, fieldnames=fieldnames)
writer.writeheader()

#解析&存储
book_infos = extract_bookinfos(html)
for book_info in book_infos:
    writer.writerow(book_info)
    
#关闭csv
csvf.close()

In [19]:
#复杂数据
import requests

def download(link, file):
    """
    下载pdf报告文件
    link： pdf报告的链接
    file: 存储文件的路径（结尾有.pdf）
    """
    resp = requests.get(link)
    #获取到二进制数据
    binarydata = resp.content
    #以二进制形式将数据流存入fname中
    with open(file, 'wb') as f:
        f.write(binarydata) 
 
link = 'http://www.sse.com.cn/disclosure/listedinfo/announcement/c/2019-10-30/600000_2019_3.pdf'
file = 'data/pdfs/{pdfname}'.format(pdfname=link.split('/')[-1])
download(link, file)        

In [ ]:
#5.汇总爬虫
import requests
import csv
import re
from pyquery import PyQuery


def generate_urls(limit=50):
    """
    生成网站的url列表
    limit: 最大可采集页数，默认是50。豆瓣小说允许的最大采集不得超过50页。
    """
    urls = []
    template = 'https://book.douban.com/tag/小说?start={param}&type=T'
    for p in range(1, limit+1):
        url = template.format(param=(p-1)*20)
        urls.append(url)   
    return urls

def get_html(url):
    """
    获取url对应的html数据(网页源代码字符串数据)，返回html数据
    url: 网页的url
    """
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.122 Safari/537.36'}
    resp = requests.get(url, headers=headers)
    html = resp.text
    return html

def extract_bookinfos(html):
    """
    从网页中抽取出书的相关信息，以字典列表返回
    html: 网页字符串数据
    """
    bookinfos = []
    doc = PyQuery(html)
    for book in doc.items('.subject-item'):
        try:
            img = book('.pic a img').attr('src')
            book_name = book('.info h2 a').text()
            info = book('.info .pub').text().split('/')
            price = info[-1]
            year = info[-2]
            pub = re.sub('\s', '', info[-3])
            authors = ''.join(info[:-3])
            rating = book('.rating_nums').text()
            #rate_people = book('span[class=pl]')
            raw_people = book('.pl').text()
            rate_people = re.findall('\d+', raw_people)[0]
            desc = book('.info p').text()                      
            book_info = {'img':img,
                         'book_name':book_name,
                         'price':price,
                         'year':year,
                         'pub':pub,
                         'authors':authors,
                         'rating': rating,
                         'rate_people':rate_people,
                         'desc':desc
                        }
            bookinfos.append(book_info)
        except:
            pass
        
    return bookinfos
#主函数
def main(file, limit=50):
    """
    豆瓣小说爬虫
    file: csv文件路径
    limit: 页面最大采集数。豆瓣小说允许我们limit<=50，所以本爬虫默认limit=50
    """
    print('准备采集豆瓣小说！')
    #新建csv
    csvf = open(file, 'a+', encoding='utf-8', newline='')
    fieldnames = ['book_name', 'price', 'year', 'pub', 'authors', 'rating', 'rate_people', 'desc', 'img']
    writer = csv.DictWriter(csvf, fieldnames=fieldnames)
    writer.writeheader()
    #生成网址urls
    urls = generate_urls(limit)
    for url in urls:
        print('正在采集: {url}'.format(url=url))
        #访问，获取url的html数据
        html = get_html(url)
        #从html中抽取出书的字典信息
        book_infos = extract_bookinfos(html)
        for book_info in book_infos:
            #将书的字典信息写入csv
            writer.writerow(book_info)
    #关闭csv
    csvf.close()
    print('豆瓣小说采集完毕！')
     
main(file = 'data/books.csv', limit=10)

In [20]:
#复杂网址
import json

def generate_url(stockcode, beginDate, endDate):
    """
    生成年报搜索页的url
    注意，开始日期与结束日期之间要小于三年
    stockcode： 股票代码字符串 比如'600000'
    beginDate： 开始日期字符串，如'2017-10-05'
    endDate： 结束日期字符串，如'2020-04-05'
    """
    template = 'http://query.sse.com.cn/security/stock/queryCompanyBulletin.do?jsonCallBack=jsonpCallback10470&isPagination=true&productId={stockcode}&securityType=0101%2C120100%2C020100%2C020200%2C120200&reportType2=DQBG&reportType=&beginDate={beginDate}&endDate={endDate}&pageHelp.pageSize=25&pageHelp.pageCount=50&pageHelp.pageNo=1&pageHelp.beginPage=1&pageHelp.cacheSize=1&pageHelp.endPage=5&_=1588153366632'
    url = template.format(stockcode=stockcode, beginDate=beginDate, endDate=endDate)
    return url


def get_json(url):
    """
    获取url（报告列表页的url）对应的json数据
    返回raw_json
    """
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.122 Safari/537.36',
              'Referer': 'http://www.sse.com.cn/disclosure/listedinfo/regular/'}

    resp = requests.get(url, headers=headers)#, cookies=cookies)

    raw_json = resp.text
    return raw_json


def extract_pdf_links(raw_json):
    """
    从raw_json中抽取出pdf报告的链接link，返回的是link列表
    """
    links = []
    jsonstring = raw_json.replace('jsonpCallback10470(', '')[:-1]
    raw_dict_data = json.loads(jsonstring)
    results = raw_dict_data['result']
    for result in results:
        link = 'http://www.sse.com.cn'+result['URL']
        links.append(link)
    return links


def download(link, file):
    """
    下载pdf报告文件
    link： pdf报告的链接
    file: 存储文件的路径（结尾有.pdf）
    """
    
    resp = requests.get(link)
    
    #获取到二进制数据
    binarydata = resp.content
    
    #以二进制形式将数据流存入fname中
    with open(file, 'wb') as f:
        f.write(binarydata) 
        
        
def main(stockcode, beginDate, endDate):
    """
    获取某stockcode在(beginDate,endDate)之间的所有pdf报告链接
    注意，开始日期与结束日期之间要小于三年
    stockcode： 股票代码字符串 比如'600000'
    beginDate： 开始日期字符串，如'2017-10-05'
    endDate： 结束日期字符串，如'2020-04-05'
    """
    
    print('准备采集{code}在[{begin},{end}]之间的报告'.format(code=stockcode, begin=beginDate, end=endDate))
    #获取某公司在(beginDate,endDate)之间的所有pdf报告链接
    url = generate_url(stockcode, beginDate, endDate)
    raw_json = get_json(url)
    links = extract_pdf_links(raw_json)
    
    for link in links:
        
        file = 'data/pdfs/{pdfname}'.format(pdfname=link.split('/')[-1])
        print('正在下载报告:{link}'.format(link=link))
        download(link, file)
    print('采集完毕 {code}在[{begin},{end}]之间的报告'.format(code=stockcode, begin=beginDate, end=endDate))
        
    
stockcode = '600000'  
beginDate = '2017-10-05'
endDate = '2020-04-05'
main(stockcode, beginDate, endDate) 

准备采集600000在[2017-10-05,2020-04-05]之间的报告
正在下载报告:http://www.sse.com.cn/disclosure/listedinfo/announcement/c/2019-10-30/600000_2019_3.pdf
正在下载报告:http://www.sse.com.cn/disclosure/listedinfo/announcement/c/2019-08-24/600000_2019_z.pdf
正在下载报告:http://www.sse.com.cn/disclosure/listedinfo/announcement/c/2019-08-24/600000_2019_zzy.pdf
正在下载报告:http://www.sse.com.cn/disclosure/listedinfo/announcement/c/2019-04-30/600000_2019_1.pdf
正在下载报告:http://www.sse.com.cn/disclosure/listedinfo/announcement/c/2019-03-26/600000_2018_n.pdf
正在下载报告:http://www.sse.com.cn/disclosure/listedinfo/announcement/c/2019-03-26/600000_2018_nzy.pdf
正在下载报告:http://www.sse.com.cn/disclosure/listedinfo/announcement/c/2018-10-31/600000_2018_3.pdf
正在下载报告:http://www.sse.com.cn/disclosure/listedinfo/announcement/c/2018-08-30/600000_2018_z.pdf
正在下载报告:http://www.sse.com.cn/disclosure/listedinfo/announcement/c/2018-08-30/600000_2018_zzy.pdf
正在下载报告:http://www.sse.com.cn/disclosure/listedinfo/announcement/c/2018-04-28/600000_2017_n.pdf
正在下载

In [21]:
#利用爬虫下载图片、视频、音频、文件
import requests

def download(url, file):
    """
    下载多媒体及文件
    url： 多媒体文件链接（结尾有文件格式名）
    file: 存储文件的路径（结尾有文件格式名）
    """
    resp = requests.get(url)
    #获取到二进制数据
    binarydata = resp.content
    #以二进制形式将数据流存入fname中
    with open(file, 'wb') as f:
        f.write(binarydata)  
#上述为未反爬，下述为避开反扒
def download(url, fname, headers=None, cookies=None):
    """
    下载多媒体及文件
    url： 多媒体文件链接（结尾有文件格式名）
    fname: 存储文件的路径（结尾有文件格式名）
    headers: 伪装请求头user-agent
    cookies: 待采集网站的cookies
    """
    
    resp = requests.get(url, headers=headers, cookies=cookies)
    
    #获取到二进制数据
    binarydata = resp.content
    
    #以二进制形式将数据流存入fname中
    with open(fname, 'wb') as f:
        f.write(binarydata) 

In [ ]:
# 图片
img_url = 'xx.jpg'
download(url=img_url, file='data/test.jpg')

In [ ]:
# 音频
music_url = 'xx.m4a'
download(url=music_url, file='data/test.m4a')

In [ ]:
# 视频
video_url = 'xx.mp4'
download(url=video_url, file='data/test.mp4')

In [ ]:
# 文件
pdf_url = 'xx.pdf'
download(url=pdf_url, file='data/test.pdf')